**Table of contents**<a id='toc0_'></a>    
- 1. [Analyse de Sentiment Basée sur des Tweets - Modèle Baseline](#toc1_)    
  - 1.1. [Préparation des Données](#toc1_1_)    
  - 1.2. [Vectorisation des Tweets](#toc1_2_)    
  - 1.3. [Entraînement et Évaluation du Modèle](#toc1_3_)    
  - 1.4. [Comparaison des Résultats](#toc1_4_)    
    - 1.4.1. [Comparaison des Performances entre TF-IDF et Count Vectorizer](#toc1_4_1_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Analyse de Sentiment Basée sur des Tweets - Modèle Baseline](#toc0_)

Ce notebook présente un modèle de base pour l'analyse de sentiment de tweets en utilisant la **régression logistique**. Nous explorerons deux méthodes de vectorisation du texte : **TF-IDF** et **Count Vectorizer**, afin de convertir le texte en caractéristiques numériques exploitables par le modèle. 

Les principaux objectifs sont :
- Préparer les données textuelles en utilisant des méthodes de vectorisation.
- Entraîner un modèle de régression logistique sur les tweets vectorisés.
- Suivre et évaluer les performances du modèle en utilisant **MLFlow** pour chaque méthode de vectorisation.

In [1]:
# Manipulation de données et calculs
import pandas as pd
import joblib
import os
import time

# Bibliothèques pour la vectorisation et la transformation des textes
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Modèle de régression logistique pour la classification
from sklearn.linear_model import LogisticRegression

# Bibliothèques pour la division des données et les métriques d'évaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Bibliothèque pour le suivi des expérimentations
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

## 1.1. <a id='toc1_1_'></a>[Préparation des Données](#toc0_)

Nous chargeons le dataset prétraité, qui contient les tweets et leur sentiment associé.

In [2]:
# Charger les données prétraitées
df = pd.read_csv('../data/processed_tweets.csv')
X_text = df['text']
y = df['sentiment']

In [3]:
df.head()

,sentiment,text
0,0,awww thats bummer shoulda got david carr third...
1,0,upset cant update facebook texting might cry r...
2,0,dived many time ball managed save 50 rest go b...
3,0,whole body feel itchy like fire
4,0,behaving im mad cant see


In [4]:
# Initialiser MLFlow et Démarrer une Expérimentation
mlflow.set_experiment("Sentiment_Analysis_Baseline_Model")

<Experiment: artifact_location='file:///e:/Openclassroom/AI_Engineer/Projet_07/work_folder/mlruns/904868608763176366', creation_time=1732026600769, experiment_id='904868608763176366', last_update_time=1732026600769, lifecycle_stage='active', name='Sentiment_Analysis_Baseline_Model', tags={}>

## 1.2. <a id='toc1_2_'></a>[Vectorisation des Tweets](#toc0_)

Nous utilisons **TF-IDF** et **Count Vectorizer** pour créer des représentations numériques des tweets. TF-IDF pèse l’importance des mots en fonction de leur fréquence, tandis que Count Vectorizer compte les occurrences de chaque mot. Nous limiterons les caractéristiques à un maximum de 5000 mots pour optimiser le modèle.

In [5]:
# Créer les Features avec TF-IDF et Count Vectorizer
vectorizer_tfidf = TfidfVectorizer(max_features=5000)
vectorizer_count = CountVectorizer(max_features=5000)

# Vectoriser les textes en utilisant TF-IDF
X_tfidf = vectorizer_tfidf.fit_transform(X_text)
# Vectoriser les textes en utilisant Count Vectorizer
X_count = vectorizer_count.fit_transform(X_text)

# Diviser les données en jeu d'entraînement et de test pour chaque méthode
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
X_train_count, X_test_count, _, _ = train_test_split(X_count, y, test_size=0.2, random_state=42)

## 1.3. <a id='toc1_3_'></a>[Entraînement et Évaluation du Modèle](#toc0_)

Dans cette section, nous entraînons un modèle de **régression logistique** pour chaque méthode de vectorisation. Nous utilisons MLFlow pour suivre les expérimentations et enregistrer les métriques clés, telles que la précision, le F1-score et le ROC-AUC, afin de comparer les performances des deux méthodes de vectorisation.

In [ ]:
# Créer un dossier pour sauvegarder les modèles
output_dir = "../saved_models"
os.makedirs(output_dir, exist_ok=True)

# Nommer l'expérience dans MLFlow
mlflow.set_experiment("Sentiment_Analysis_Baseline_Model")

# Entraîner les modèles et suivre les expérimentations
for vectorizer_name, X_train, X_test in [("TF-IDF", X_train_tfidf, X_test_tfidf), ("Count Vectorizer", X_train_count, X_test_count)]:
    with mlflow.start_run():
        # Mesurer le temps d’entraînement
        start_time = time.time()

        # Entraîner le modèle
        model = LogisticRegression()
        model.fit(X_train, y_train)

        # Calculer le temps écoulé
        elapsed_time = time.time() - start_time

        # Prédictions et Probabilités
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        # Évaluation des performances
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        report = classification_report(y_test, y_pred, output_dict=True)

        # Loguer les paramètres et métriques dans MLFlow
        mlflow.log_param("vectorizer", vectorizer_name)
        mlflow.log_param("model_type", "Logistic Regression")
        mlflow.log_param("max_features", 5000)

        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("roc_auc", roc_auc)
        mlflow.log_metric("precision", report["weighted avg"]["precision"])
        mlflow.log_metric("recall", report["weighted avg"]["recall"])
        mlflow.log_metric("f1-score", report["weighted avg"]["f1-score"])
        mlflow.log_metric("training_time_seconds", elapsed_time)

        # Exemple d'entrée pour MLFlow
        input_example = X_test[0].toarray()  # Convertir le premier tweet vectorisé en format array

        # Signature du modèle
        signature = infer_signature(X_test, model.predict(X_test))

        # Enregistrer le modèle avec signature et exemple dans MLFlow
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path=f"baseline_model_{vectorizer_name.lower().replace(' ', '_')}",
            input_example=input_example,
            signature=signature
        )

        # Enregistrer le modèle et le vectorizer dans des fichiers locaux
        model_file = os.path.join(output_dir, f"logistic_regression_{vectorizer_name.lower().replace(' ', '_')}.pkl")
        vectorizer_file = os.path.join(output_dir, f"{vectorizer_name.lower().replace(' ', '_')}_vectorizer.pkl")

        joblib.dump(model, model_file)
        joblib.dump(vectorizer_tfidf if vectorizer_name == "TF-IDF" else vectorizer_count, vectorizer_file)

        # Afficher les informations
        print(f"Modèle enregistré sous : {model_file}")
        print(f"Vectorizer enregistré sous : {vectorizer_file}")
        print(f"Modèle enregistré dans MLFlow sous le chemin : baseline_model_{vectorizer_name.lower().replace(' ', '_')}")
        print(f"Results for {vectorizer_name}:")
        print(f"Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}, Training Time: {elapsed_time:.2f} seconds")
        print("Classification Report:\n", classification_report(y_test, y_pred))

Modèle enregistré sous : ../saved_models\logistic_regression_tf-idf.pkl
Vectorizer enregistré sous : ../saved_models\tf-idf_vectorizer.pkl
Modèle enregistré dans MLFlow sous le chemin : baseline_model_tf-idf
Results for TF-IDF:
Accuracy: 0.7750, ROC-AUC: 0.8551, Training Time: 2.62 seconds
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.76      0.77    158794
           1       0.77      0.79      0.78    159876

    accuracy                           0.78    318670
   macro avg       0.78      0.77      0.77    318670
weighted avg       0.78      0.78      0.77    318670



Modèle enregistré sous : ../saved_models\logistic_regression_count_vectorizer.pkl
Vectorizer enregistré sous : ../saved_models\count_vectorizer_vectorizer.pkl
Modèle enregistré dans MLFlow sous le chemin : baseline_model_count_vectorizer
Results for Count Vectorizer:
Accuracy: 0.7747, ROC-AUC: 0.8492, Training Time: 4.20 seconds
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.74      0.77    158794
           1       0.76      0.81      0.78    159876

    accuracy                           0.77    318670
   macro avg       0.78      0.77      0.77    318670
weighted avg       0.78      0.77      0.77    318670

